In [1]:
import pandas as pd
import numpy as np

In [22]:
temp_df_pt1 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_1.csv')
temp_df_pt2 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_2.csv')

# concatenate dataframes
# gonna turn all the wrong years into -1
frames = [temp_df_pt1, temp_df_pt2]
temp_df = pd.concat(frames, sort=False)

tree_df = pd.read_csv('../tree_data.csv')
tree_df['year'] = tree_df['year'].apply(lambda x: int(x) if str(x).isdigit() else -1)
tree_df.head()

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group
0,969,164.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
1,970,188.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
2,971,139.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
3,972,173.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
4,973,221.0,51.75041,-0.34223,NaN,Quercus spp. L.,0


In [23]:
tree_df['tree_species'].value_counts()

Pseudotsuga menziesii (Mirb.) Franco    214694
Pinus sylvestris L.                     152902
Tsuga mertensiana (Bong.) Carriere      122337
Pinus ponderosa Douglas ex C. Lawson    112764
Picea glauca (Moench) Voss               85047
                                         ...  
Previa maxima                               60
Vitex keniensis Turr                        56
Quercus pubescens Willd.                    51
Ulmus spp. L.                               46
Pistacia lentiscus  L.                      30
Name: tree_species, Length: 246, dtype: int64

In [24]:
# How many have bad years
print(tree_df.shape)
tree_df[tree_df['year'] == -1].shape

(2062155, 7)


(120638, 7)

In [25]:
def get_5x5_grid(og_lat, og_lon, base=2.5):
    lat1 = base * round((og_lat-2.5)/base)
    lat2 = base * round((og_lat+2.5)/base)

    lon1 = base * round((og_lon-2.5)/base)
    lon2 = base * round((og_lon+2.5)/base)

    if lat2-2.5 < og_lat:
        lat2 += 2.5
        lat1 += 2.5
    else:
        lat2 -= 2.5
        lat1 -= 2.5

    if lon2-2.5 < og_lon:
        lon2 += 2.5
        lon1 += 2.5
    else:
        lon2 -= 2.5
        lon1 -= 2.5

    if (lat1 % 5 )== 0 or (lat2 % 5 )== 0:
        if lat1 < 0:
            lat1 += 2.5
            lat2 += 2.5
        else: 
            lat1 -= 2.5
            lat2 -= 2.5

    if (lon1 % 5 )== 0 or (lon2 % 5 )== 0:
        lon1 += 2.5
        lon2 += 2.5

    if (lat1 == -92.5 and lat2 == -87.5):
        lat1 = -87.5
        lat2 = -82.5
    elif (lat1 == 87.5 and lat2 == 92.5):
        lat1 = 82.5
        lat2 = 87.5
    
    if (lon1 == 0.0 and lon2 == 5.0):
        lon1 = 2.5
        lon2 = 7.5
    elif (lon1 == 355.0 and lon2 == 360.0):
        lon1 = 352.5
        lon2 = 357.5

    return lat1, lat2, lon1, lon2

In [26]:
temp_df['year'] = temp_df['time'].apply(lambda x: str(x)[0:4])
temp_df = temp_df.dropna(subset=['anom'])
temp_annual_df =  temp_df.groupby(['year', 'lat', 'lon']).agg({'anom': ['mean']}).reset_index()

In [27]:
# given an observation row, return the row with the correct 5x5 temperature anomaly

def get_gridded_temp_anom(lat, lon, year):
    lat1, lat2, lon1, lon2 = get_5x5_grid(lat, lon)
    print(lat1, lat2, lon1, lon2)
    temp_df_for_lat_lon = temp_df[((temp_df['lat'] == lat1) | (temp_df['lat'] == lat2)) &
                                  ((temp_df['lon'] == lon1) | (temp_df['lon'] == lon2)) &
                                  (temp_df['year'] == str(year))]
    annual_temp_df_for_lat_lon = temp_df_for_lat_lon.groupby(['year']).agg({'anom': ['mean']}).reset_index()
    if len(annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()) > 0:
        temp_anom = annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()[0]
        return temp_anom


In [28]:
# run it on the data

tree_df_temp = tree_df[tree_df['year'] > 1879]
tree_df_temp = tree_df_temp[tree_df_temp['year'] < 2021]
tree_df_temp = tree_df_temp[tree_df_temp['tree_species']
                            == 'Ulmus spp. L.']
tree_df_temp['temp_anom'] = tree_df_temp.apply(lambda x: get_gridded_temp_anom(x['lat'], x['long'], x['year']), axis=1)
tree_df_temp


47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5
47.5 52.5 7.5 12.5


,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group,temp_anom
256981,1946,176.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.245835
256982,1947,146.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.141897
256983,1948,93.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,0.162076
256984,1949,106.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,0.367866
256985,1950,87.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.022357
256986,1951,126.5,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.073660
256987,1952,98.0,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.667542
256988,1953,119.5,47.5828,9.2742,507.0,Ulmus spp. L.,0,0.110018
256989,1954,110.5,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.945114
256990,1955,119.5,47.5828,9.2742,507.0,Ulmus spp. L.,0,-0.969283
